In [1]:
from helper_functions import xor_hex_strings, single_byte_xor, repeating_key_xor, xor_bytes, hamming_distance, output_repeated_block, has_repeated_blocks, find_block
from file_loading import load_file_as_bytes, load_file_as_b64
from conversions import int2bytes, hex2bytes, string2bytes, hex2b64, string2dict
from symmetric_encryption import is_ecb_mode, aes128_ecb_encrypt, aes128_ecb_decrypt, aes128_cbc_encrypt, aes128_cbc_decrypt, determine_block_size
from padding import apply_pkcs_7, remove_pkcs_7
# 
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import base64
import secrets

## **Challenge 9: Implement PKCS#7 padding**

In [2]:
apply_pkcs_7(b"YELLOW SUBMARINE", 20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'

## **Challenge 10: Implement CBC mode**

In [3]:
key = secrets.token_bytes(16)
iv = secrets.token_bytes(16)
text = b'YELLOW SUBMARINE YELLOW SUBMARIN'

aes128_cbc_decrypt(aes128_cbc_encrypt(text, key, iv), key, iv)

b'YELLOW SUBMARINE YELLOW SUBMARIN'

In [4]:
file_bytes = load_file_as_b64("challenge_data/10.txt", remove_newlines=True)

key = b'YELLOW SUBMARINE'
IV = b'\x00' * 16
aes128_cbc_decrypt(file_bytes, key, IV).decode('utf-8')

"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take a

## **Challenge 11: An ECB/CBC detection oracle**

In [5]:
def encryption_oracle_11(plaintext: bytes) -> bytes:
    """Encrypts a `plaintext` with AES-128 in either ECB or CBC mode. The plaintext is padded with PKCS#7 padding
    and a random prefix and suffix are added to the plaintext. The key and IV are chosen at random. The function
    returns a tuple of the mode used and the cyphertext. 
    
    `encrypt(before_bytes + plaintext + after_bytes)`
    """

    key = secrets.token_bytes(16)  # for AES-128
    iv = secrets.token_bytes(16)  # for AES-128

    before_bytes = secrets.token_bytes(secrets.choice([5, 6, 7, 8, 9, 10]))
    after_bytes = secrets.token_bytes(secrets.choice([5, 6, 7, 8, 9, 10]))

    modified_plaintext = apply_pkcs_7(before_bytes + plaintext + after_bytes, 16)

    if secrets.choice([True, False]):
        print("ECB")
        return aes128_ecb_encrypt(modified_plaintext, key)
    else:
        print("CBC")
        return aes128_cbc_encrypt(modified_plaintext, key, iv)

In [6]:
for _ in range(5):
    ecb_mode = is_ecb_mode(encryption_oracle_11, 16)
    if ecb_mode:
        print("ECB", "\n")
    else:
        print("CBC", "\n")

ECB
ECB 

CBC
CBC 

ECB
ECB 

ECB
ECB 

CBC
CBC 



## **Challenge 12: Byte-at-a-time ECB decryption (Simple)**

In [7]:
# Assume that key is unknown but same for all encryptions
encryption_oracle_12_key = secrets.token_bytes(16)

In [8]:
def encryption_oracle_12(plaintext: bytes) -> bytes:
    """Encrypts a `plaintext` with AES-128 in ECB mode. The plaintext is padded with PKCS#7 padding.
    There is an unknown string appended to the plaintext. The key is chosen at random. The function
    returns the cyphertext."""
    unknown_string = base64.decodebytes(b"Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")
    return aes128_ecb_encrypt(apply_pkcs_7(plaintext + unknown_string, 16), encryption_oracle_12_key)

In [9]:
# step 1: find the block length
block_size, offset = determine_block_size(encryption_oracle_12)
print(f"Block length: {block_size}")
print(f"Padding offset (bytes to add so that there is no padding): {offset}")

# step 2: determine if the encryption is ECB or CBC
print(f"Is this encrypted with ECB mode: {is_ecb_mode(encryption_oracle_12, block_size)}")

# step 3: make dictionary of all possible last bytes by brute forcing the block by block
plaintext = b''

def gen_lookup_table(plaintext: bytes, block_size: int, oracle: callable) -> dict:
    """Generates a lookup table for the last byte of the plaintext.
    The lookup table maps the cyphertext to the last byte of the plaintext."""
    cypher2plain = {}                                   # maps cyphertext to plaintext
    w = plaintext[(1-block_size):]                      # window of last block_size - 1 bytes of plaintext
    pad = b'A' * (block_size - len(w) - 1)              # used to pad the first block *ONLY*
    for i in range(256):                                # iterate through all possible final bytes
        p = pad + w + int2bytes(i)                      # generate plaintext to encrypt
        cy = oracle(p)                                  # encrypt the plaintext
        cypher2plain[cy[:block_size]] = int2bytes(i)    # add to dictionary
    return cypher2plain


def padded_encryption(plaintext: bytes, block_size: int, oracle: callable) -> bytes:
    """Pad and encrypt the plaintext. The padding is done so that a full block only has a single unknown byte."""
    pad = b'A' * ((block_size - len(plaintext) - 1) % block_size)
    return oracle(pad)


for char_i in range(10000): # 10000 is an arbitrary large number
    cyphertext = padded_encryption(plaintext, block_size, encryption_oracle_12)

    # When the full plaintext is found, the last block will not be in the dictionary
    block_num = char_i // block_size    # current block number
    selected_cypher = cyphertext[block_num*block_size:(block_num + 1)*block_size]
    try:
        cypher2plain = gen_lookup_table(plaintext, block_size, encryption_oracle_12)
        plaintext += cypher2plain[selected_cypher]
    except KeyError: 
        break


plaintext.decode('utf-8')

Block length: 16
Padding offset (bytes to add so that there is no padding): 5
Is this encrypted with ECB mode: True


"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"

## **Challenge 13: ECB cut-and-paste**

In [10]:
encryption_oracle_13_key = secrets.token_bytes(16)

In [11]:
def profile_for(email: str):
    """Returns a profile for an email address. The profile is a string of the form:
    `email=<email>&uid=<10 random digits>&role=user`
    """
    email = email.replace("&", "").replace("=", "") # eat the characters (yum yum!)
    uid = 1000 + secrets.randbelow(10**4 - 1000)    # 1000 <= uid < 10**4
    object = f"email={email}&uid={uid}&role=user"   # the object to be encrypted
    padded_object = apply_pkcs_7(object.encode('utf-8'), 16)            # pad the object
    return aes128_ecb_encrypt(padded_object, encryption_oracle_13_key)  # encrypt the object

def decrypt_profile(profile: bytes) -> dict:
    """Decrypts a profile and returns a dictionary of the form:
    `{'email': <email>, 'uid': <uid>, 'role': <role>}`
    """
    decrypted_profile = aes128_ecb_decrypt(profile, encryption_oracle_13_key)
    decrypted_profile = remove_pkcs_7(decrypted_profile)
    decrypted_profile = decrypted_profile.decode('utf-8')
    return string2dict(decrypted_profile)

In [12]:
block_size = 16

# 10 A's push the admin string into its own block
a = 'A' * 10

# admin padding with correct padding bytes
admin_str = apply_pkcs_7(b'admin', 16).decode('utf-8') 

# get the encrypted profile for the admin string
admin_encrypted_profile = profile_for(a + admin_str)

# extract the encrypted admin string
admin_bytes = admin_encrypted_profile[block_size:2*block_size]

# 1. get the encrypted profile for the user
# 2. remove the last block
# 3. append the admin bytes
hacker_encrypted_profile = profile_for("hackerman69")
hacker_encrypted_profile = hacker_encrypted_profile[:-block_size]
hacker_encrypted_profile += admin_bytes

print("Successfully created an encrypted profile for the admin user!")
print(decrypt_profile(hacker_encrypted_profile))

Successfully created an encrypted profile for the admin user!
{'email': 'hackerman69', 'uid': '4018', 'role': 'admin'}


## **Challenge 14: Byte-at-a-time ECB decryption (Harder)**

In [13]:
encryption_oracle_14_key = secrets.token_bytes(16)

In [14]:
def encryption_oracle_14(plaintext: bytes) -> bytes:
    """Encrypts a `plaintext` with AES-128 in ECB mode. The plaintext is padded with PKCS#7 padding.
    `AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)`
    """
    max_num_bytes = 32
    rand_bytes = secrets.token_bytes(secrets.randbelow(max_num_bytes))
    unknown_string = base64.decodebytes(b"Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK")
    return aes128_ecb_encrypt(apply_pkcs_7(rand_bytes + plaintext + unknown_string, 16), encryption_oracle_14_key)

In [15]:
# step 1: find the block length
block_size, offset = determine_block_size(encryption_oracle_14)
print(f"Block length: {block_size}")
print(f"Padding offset (bytes to add so that there is no padding): {offset}")

# step 2: determine if the encryption is ECB or CBC
print(f"Is this encrypted with ECB mode: {is_ecb_mode(encryption_oracle_14, block_size)}")

plaintext = b''
def altered_oracle(plaintext: bytes, block_size: int) -> bytes:
    """The oracle with the altered plaintext"""
    
    # use flag to determine when the inserted plaintext starts
    flag = b'F' * block_size
    cy_flag = output_repeated_block(encryption_oracle_14(3*flag), block_size)
    while True:
        cy = encryption_oracle_14(flag + plaintext)
        i = find_block(cy, cy_flag, block_size)
        if i:
            return cy[i+block_size:]

# alter the oracle so that the inserted plaintext starts at the beginning of a block
oracle_14_no_rand_bytes = lambda x: altered_oracle(x, block_size)
for char_i in range(10000): # 10000 is an arbitrary large number
    cyphertext = padded_encryption(plaintext, block_size, oracle_14_no_rand_bytes)

    block_num = char_i // block_size    # current block number
    selected_cypher = cyphertext[block_num*block_size:(block_num + 1)*block_size]
    try:
        cypher2plain = gen_lookup_table(plaintext, block_size, oracle_14_no_rand_bytes)
        plaintext += cypher2plain[selected_cypher]
    except KeyError: 
        break

    print(plaintext)


Block length: 16
Padding offset (bytes to add so that there is no padding): 1
Is this encrypted with ECB mode: True
b'R'
b'Ro'
b'Rol'
b'Roll'
b'Rolli'
b'Rollin'
b"Rollin'"
b"Rollin' "
b"Rollin' i"
b"Rollin' in"
b"Rollin' in "
b"Rollin' in m"
b"Rollin' in my"
b"Rollin' in my "
b"Rollin' in my 5"
b"Rollin' in my 5."
b"Rollin' in my 5.0"
b"Rollin' in my 5.0\n"
b"Rollin' in my 5.0\nW"
b"Rollin' in my 5.0\nWi"
b"Rollin' in my 5.0\nWit"
b"Rollin' in my 5.0\nWith"
b"Rollin' in my 5.0\nWith "
b"Rollin' in my 5.0\nWith m"
b"Rollin' in my 5.0\nWith my"
b"Rollin' in my 5.0\nWith my "
b"Rollin' in my 5.0\nWith my r"
b"Rollin' in my 5.0\nWith my ra"
b"Rollin' in my 5.0\nWith my rag"
b"Rollin' in my 5.0\nWith my rag-"
b"Rollin' in my 5.0\nWith my rag-t"
b"Rollin' in my 5.0\nWith my rag-to"
b"Rollin' in my 5.0\nWith my rag-top"
b"Rollin' in my 5.0\nWith my rag-top "
b"Rollin' in my 5.0\nWith my rag-top d"
b"Rollin' in my 5.0\nWith my rag-top do"
b"Rollin' in my 5.0\nWith my rag-top dow"
b"Rollin' in 

## **Challenge 15: PKCS#7 padding validation**

In [16]:
remove_pkcs_7(b"ICE ICE BABY\x04\x04\x04\x04")

b'ICE ICE BABY'

In [17]:
try:
    remove_pkcs_7(b"ICE ICE BABY\x05\x05\x05\x05")
except ValueError:
    print("Successfully raised a ValueError!")

Successfully raised a ValueError!


In [18]:
try:
    remove_pkcs_7(b"ICE ICE BABY\x01\x02\x03\x04")
except ValueError:
    print("Successfully raised a ValueError!")

Successfully raised a ValueError!
